In [1]:
# Read all the libraries which we're going to use in our bot & Load Data
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

# Just disables the warning, doesn't take advantage of AVX/FMA to run faster
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Numpy is a matrices libraries
import numpy as np
from sklearn.utils import shuffle

## Import tensorflow for using Keras, instead.
import tensorflow as tf
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

## Import Natural Language Processing toolkit
import nltk
# nltk.download()    # for missing language interpretable
from nltk.stem import WordNetLemmatizer


physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(physical_devices)}")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Import jason files and pickles to read our training datasets
import json
import pickle

# create an instance from nltk
lemmatizer = WordNetLemmatizer()

# opens & read the intents JSON file dataset for data training
intents_file = open(os.path.abspath('intents.json')).read()
intents = json.loads(intents_file)

In [2]:
# Data pre-processing
words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)        
        # adding document in the corpus
        documents.append((word, intent['tag']))
        # add the formula tag to the class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# --------------------------------------------------------------------------------------------------------------------------------------------------------------#
# Now, we're looking for reduce all the canonical words – so we can reduce the number of total words in our vocabulary.
# Lemmatization, is reducing duplicates words that are similar to each other. (e.g. play, playing, plays, played)
## lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(list(set(words)))
## sort classes
classes = sorted(tuple(set(classes)))
## document = combination between patterns and intents
print(len(documents), "documents")
## classes = intents
print(len(classes), "classes", classes)
## words = all words, vocabulary
print(len(words), "unique lemmatize words", words)

pickle.dump(words, open(os.path.abspath('words.pkl'), 'wb'))
pickle.dump(classes, open(os.path.abspath('classes.pkl'), 'wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatize words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [87]:
# Create the training data
training = []
# Create empty array for the output
output_zeros = np.zeros(len(classes)).astype(int)
# training set, bag of words for everysentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the patterns – intents
    word_patterns = doc[0]
    # lemmatize each word – create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # append in the bag of words array with 1, if the word is found in the current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    # output is a '0' for each tag, and '1' for current tag (for each pattern)
    output_row = list(output_zeros)   # we assigned the empty matrix to new variables to avoid the accumulation that caused by using global variable in the loop. 
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# # shuffle the features and make numpy array
shuffle(training)
training = np.asarray(training)

# create training and testing set. X – (a.k.a patterns, entities, words), Y – (a.k.a intents, classes)
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data is created!")


Training data is created!


In [88]:
# Build the deep neural network model
model = Sequential([
                    Dense(128, input_shape=(len(train_x[0]), ), activation='relu'),
                    Dropout(0.5), # reduce the model rate to 0.5 to reduce the overfitting
                    Dense(64, activation='relu'),
                    Dropout(0.5), # reduce the model rate to 0.5 to reduce the overfitting
                    Dense(len(train_y[0]), activation='softmax'),
            ]) 

# Initialize an optimizer
## Stochastic gradient descent (SGD) with Nesterov Accelerated Gradient gives good result and speed to the model
sgd_nag = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# Start to compile the model to prepare the training dataset for the fitting process
model.compile(optimizer=sgd_nag, loss='categorical_crossentropy', metrics=['accuracy'])

In [89]:
# Fitting or training the model
hist = model.fit(x=np.array(train_x), y=np.array(train_y) ,batch_size=10, epochs=200, verbose=2)
hist

Epoch 1/200
5/5 - 1s - loss: 2.2241 - accuracy: 0.1915
Epoch 2/200
5/5 - 0s - loss: 2.2729 - accuracy: 0.0851
Epoch 3/200
5/5 - 0s - loss: 2.2155 - accuracy: 0.1489
Epoch 4/200
5/5 - 0s - loss: 2.1352 - accuracy: 0.2340
Epoch 5/200
5/5 - 0s - loss: 2.1224 - accuracy: 0.2128
Epoch 6/200
5/5 - 0s - loss: 2.1125 - accuracy: 0.1702
Epoch 7/200
5/5 - 0s - loss: 2.0318 - accuracy: 0.2766
Epoch 8/200
5/5 - 0s - loss: 2.0225 - accuracy: 0.2766
Epoch 9/200
5/5 - 0s - loss: 1.9471 - accuracy: 0.5106
Epoch 10/200
5/5 - 0s - loss: 1.8521 - accuracy: 0.4681
Epoch 11/200
5/5 - 0s - loss: 1.7265 - accuracy: 0.4681
Epoch 12/200
5/5 - 0s - loss: 1.7610 - accuracy: 0.5319
Epoch 13/200
5/5 - 0s - loss: 1.7060 - accuracy: 0.4681
Epoch 14/200
5/5 - 0s - loss: 1.7326 - accuracy: 0.4468
Epoch 15/200
5/5 - 0s - loss: 1.5910 - accuracy: 0.5745
Epoch 16/200
5/5 - 0s - loss: 1.5066 - accuracy: 0.6383
Epoch 17/200
5/5 - 0s - loss: 1.4887 - accuracy: 0.5319
Epoch 18/200
5/5 - 0s - loss: 1.2929 - accuracy: 0.7872
E

In [90]:
# Saving the model
model.save(os.path.abspath('chatPyMe.h5'), hist)
print('Model Created!')

Model Created!
